In [22]:
import torch
from torch.nn.functional import cosine_similarity
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer, AutoModel

from tqdm.notebook import tqdm
import json
from pathlib import Path
import re
import random
import os
import numpy as np
import pandas as pd

# 1) Importing query and collection data

In [23]:
PATH_COLLECTION_DATA = 'subtask_4b/subtask4b_collection_data.pkl' 
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [24]:
PATH_QUERY_TRAIN_DATA = 'subtask_4b/subtask4b_query_tweets_train.tsv'
PATH_QUERY_DEV_DATA = 'subtask_4b/subtask4b_query_tweets_dev.tsv' 
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [25]:
#df_query_train.head()

In [5]:
#df_collection.head()

# 2) Running the BM25 baseline
The following code runs a BM25 baseline.


In [15]:
from rank_bm25 import BM25Okapi

In [16]:
# Create the BM25 corpus
corpus = df_collection[:][['title', 'abstract']].apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection[:]['cord_uid'].tolist()
tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
def get_top_cord_uids(query):
  text2bm25top = {}
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')
      doc_scores = bm25.get_scores(tokenized_query)
      indices = np.argsort(-doc_scores)[:100] # @k: how many docs shall the ranked list include?
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk


In [7]:
# Retrieve top50 candidates using the BM25 model

train_pkl_path = 'df_query_train_top100.pkl'
dev_pkl_path = 'df_query_dev_top100.pkl'

if not os.path.exists(train_pkl_path):
    df_query_train['bm25_topk'] = df_query_train['tweet_text'].apply(lambda x: get_top_cord_uids(x))
    df_query_train.to_pickle(train_pkl_path)
else:
    df_query_train = pd.read_pickle(train_pkl_path)

if not os.path.exists(dev_pkl_path):
    df_query_dev['bm25_topk'] = df_query_dev['tweet_text'].apply(lambda x: get_top_cord_uids(x))
    df_query_dev.to_pickle(dev_pkl_path)
else:
    df_query_dev = pd.read_pickle(dev_pkl_path)

# 3) Neural Re-Ranking

In [26]:
# load BM25 pre-ranked query dataframes
train_pkl_path = 'df_query_train_top100.pkl'
dev_pkl_path = 'df_query_dev_top100.pkl'

df_query_dev = pd.read_pickle(dev_pkl_path)
df_query_train = pd.read_pickle(train_pkl_path)

## 3.1) ColBERT w/ fine-tuned BERT

In [30]:
# get token embeddings of a specified text passage from some model
def get_token_embeddings(text, tokenizer, model, device='cpu'):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model(**inputs)
    token_embeddings = outputs.last_hidden_state.squeeze(0)
    attention_mask = inputs['attention_mask'].squeeze(0).bool()
    token_embeddings = token_embeddings[attention_mask] 
    return token_embeddings

# pre compute all the token embeddings of the documents
def build_and_save_doc_embeddings(
    docs_df,
    model_name,
    save_dir,
    max_len=512,
    device="cuda"
):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    save_path = Path("doc_embeddings_" + save_dir)
    save_path.mkdir(parents=True, exist_ok=True)

    metadata_path = save_path / "metadata.json"
    if metadata_path.exists():
        with open(metadata_path, "r") as f:
            metadata = json.load(f)
    else:
        metadata = {}

    print("Precomputing document embeddings.")
    for i, row in tqdm(docs_df.iterrows(), total=len(docs_df)):
        doc_id = row.get("cord_uid", f"doc_{i}")
        file_path = save_path / f"{doc_id}.pt"

        if file_path.exists() and doc_id in metadata:
            continue

        text = str(row.get('title', '')) + " " + str(row.get('abstract', '')) + " Authors: " + str(row.get('authors', ''))

        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=max_len)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        output = model(**inputs)
        token_embeddings = output.last_hidden_state.squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0).bool()
        token_embeddings = token_embeddings[attention_mask]

        n_tokens = token_embeddings.size(0)
        pad_len = max_len - n_tokens

        if pad_len > 0:
            padding = torch.zeros(pad_len, token_embeddings.size(1), device=device)
            token_embeddings = torch.cat([token_embeddings, padding], dim=0)
        else:
            token_embeddings = token_embeddings[:max_len]

        try:
            torch.save(token_embeddings.cuda(), file_path)
        except Exception as e:
            print(f"Error saving document {doc_id}: {e}")
            continue

        metadata[doc_id] = {
            "title": row.get("title", ""),
            "abstract": row.get("abstract", ""),
            "authors": row.get("authors", ""),
            "length": min(n_tokens, max_len),
            "path": str(file_path)
        }

    with open(metadata_path, "w") as f:
        json.dump(metadata, f)

    return metadata

# either precompute or load precomputed doc embeddings
def get_precomputed_doc_embeddings(save_name):
    def split_at_slash(s):
        if '/' in s:
            return s.split('/', 1)
        else:
            return ['', s]
        
    if not os.path.exists("doc_embeddings_" + split_at_slash(save_name)[1] + "/metadata.json"):
        metadata = build_and_save_doc_embeddings(df_collection, model_name=save_name, save_dir=save_name, device="cuda")
    else:
        with open("doc_embeddings_" + save_name + "/metadata.json", "r") as f:
            metadata = json.load(f)
    return metadata

In [31]:
# creating training dataset by getting the positive and a random negative document for each query
class ColBERTTripletDataset(Dataset):
    def __init__(self, df, metadata, tokenizer, num_negatives=1):
        self.data = []
        self.tokenizer = tokenizer
        self.metadata = metadata
        for _, row in df.iterrows():
            query = row["tweet_text"]
            pos = row["cord_uid"]
            negatives = [doc for doc in row["bm25_topk"] if doc != pos]
            if negatives:
                for _ in range(num_negatives):
                    neg = random.choice(negatives)
                    self.data.append((query, pos, neg))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# basic ColBERT scoring i.e. match matrix aggregation
def colbert_score_from_emb(q_emb, d_emb):
    q_norm = q_emb / q_emb.norm(dim=1, keepdim=True)
    d_norm = d_emb / d_emb.norm(dim=1, keepdim=True)
    sim_matrix = torch.matmul(q_norm, d_norm.T)
    max_sim_per_q = sim_matrix.max(dim=1).values
    return max_sim_per_q.sum()

# finetuning some BERT-model to get higher ColBERT-score 
# for the positive document than for the negative (per query)
def bert_finetune(save_name, MARGIN=0.5, BATCH_SIZE=8, EPOCHS=6, LR=2e-5, num_negatives=1):    
    model_name = "allenai/scibert_scivocab_uncased" # specify baseline BERT model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    metadata = get_precomputed_doc_embeddings(model_name)

    # create training triplets
    train_dataset = ColBERTTripletDataset(df_query_train, metadata, tokenizer, num_negatives)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # optimizer
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    model.train()
    model.to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    
    for epoch in range(EPOCHS):
        total_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            queries, pos_ids, neg_ids = batch
    
            inputs = tokenizer(list(queries), return_tensors='pt', padding=True, truncation=True, max_length=512)
            inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
            outputs = model(**inputs)
            q_emb_batch = outputs.last_hidden_state  # [B, L, D]
            attention_mask = inputs["attention_mask"].bool()
            q_embs = [emb[mask] for emb, mask in zip(q_emb_batch, attention_mask)]
    
            score_pos_list = []
            score_neg_list = []
    
            for i in range(len(queries)):
                d_pos_emb = torch.load(metadata[pos_ids[i]]["path"]).to(DEVICE)[:metadata[pos_ids[i]]["length"]]
                d_neg_emb = torch.load(metadata[neg_ids[i]]["path"]).to(DEVICE)[:metadata[neg_ids[i]]["length"]]
    
                q_emb = q_embs[i]
                score_pos = colbert_score_from_emb(q_emb, d_pos_emb)
                score_neg = colbert_score_from_emb(q_emb, d_neg_emb)
    
                score_pos_list.append(score_pos)
                score_neg_list.append(score_neg)
    
            score_pos_batch = torch.stack(score_pos_list)
            score_neg_batch = torch.stack(score_neg_list)
    
            loss = F.relu(MARGIN + score_neg_batch - score_pos_batch).mean()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_loss += loss.item()
    
        print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

    model.save_pretrained(save_name)
    tokenizer.save_pretrained(save_name)

In [33]:
bert_finetune("colB_sciB_marg05", MARGIN=0.5)

Precomputing document embeddings.


  0%|          | 0/7718 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/1607 [00:00<?, ?it/s]

Epoch 1 Loss: 188.6070


Epoch 2:   0%|          | 0/1607 [00:00<?, ?it/s]

...

Epoch 4 Loss: 18.8931


Epoch 5:   0%|          | 0/1607 [00:00<?, ?it/s]

Epoch 5 Loss: 16.2508


Epoch 6:   0%|          | 0/1607 [00:00<?, ?it/s]

Epoch 6 Loss: 12.9944


### Reranking:

In [34]:
def rerank(df, metadata, tokenizer, model, save_name):
    device = next(model.parameters()).device
    df[save_name + '_scores'] = [[] for _ in range(len(df))]

    doc_embeddings = {}
    for doc_id, data in metadata.items():
        emb = torch.load(data["path"], map_location="cpu")
        doc_embeddings[doc_id] = emb

    with torch.no_grad():
        for idx, row in tqdm(df.iterrows(), total=len(df)):
            tweet_text = row['tweet_text']
            pre_ranked_docs = row['bm25_topk']

            q_emb = get_token_embeddings(tweet_text, tokenizer, model).to(device)
            q_norm = q_emb / q_emb.norm(dim=1, keepdim=True)

            scores = []
            for doc in pre_ranked_docs:
                emb = doc_embeddings[doc].to(device)
                length = metadata[doc]["length"]
                d_emb = emb[:length]
                d_norm = d_emb / d_emb.norm(dim=1, keepdim=True)

                sim_matrix = torch.matmul(q_norm, d_norm.T)
                max_sim_per_q = sim_matrix.max(dim=1).values
                score = max_sim_per_q.sum().item()
                scores.append(score)

            df.at[idx, save_name + '_scores'] = scores

    def sort_docs_by_score(row):
        doc_ids = row['bm25_topk']
        scores = row[save_name + '_scores']
        sorted_docs = [doc for doc, _ in sorted(zip(doc_ids, scores), key=lambda x: x[1], reverse=True)]
        return sorted_docs

    df[save_name + '_topk'] = df.apply(sort_docs_by_score, axis=1)
    return df

In [36]:
# specify model for re-ranking
model_name = "colB_sciB_marg05"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# pre-compute embeddings
metadata = get_precomputed_doc_embeddings(model_name)

Precomputing document embeddings.


  0%|          | 0/7718 [00:00<?, ?it/s]

In [37]:
# re-rank BM25 list for dev data
df_query_dev = rerank(df_query_dev, metadata, tokenizer, model, model_name)

  0%|          | 0/1400 [00:00<?, ?it/s]

# 4) Evaluation
The following code evaluates the BM25 retrieval baseline on the query set using the Mean Reciprocal Rank score (MRR@5).

In [38]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance

In [25]:
# ---- BM25 Baseline ----
results_train = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')

print("---- BM25 Baseline ----")
print(f"Results on the train set: {results_train}")
print(f"Results on the dev set: {results_dev}")

---- BM25 Baseline ----
Results on the train set: {1: 0.5731735781529604, 5: 0.625250914183459, 10: 0.6308237901348459}
Results on the dev set: {1: 0.5657142857142857, 5: 0.616095238095238, 10: 0.6224325396825396}


In [32]:
# ---- ColBERT Re-Ranking @ Margin 0.5 ----
model_name = "colB_sciB_marg05"

results_dev = get_performance_mrr(df_query_dev, 'cord_uid', f'{model_name}_topk')
print("---- Re-Ranking Finetune: ColBERT (SciBERT) ----")
print(f"MRR@5 on dev set: {results_dev[5]}")

---- Re-Ranking Finetune-4: ColBERT (SciBERT) ----
MRR@5: 0.6806309523809524


In [39]:
# ---- ColBERT Re-Ranking @ Margin 0.6 ----
model_name = "colB_sciBERT_marg06"

results_dev = get_performance_mrr(df_query_dev, 'cord_uid', f'{model_name}_topk')
print("---- Re-Ranking Finetune: ColBERT (SciBERT) ----")
print(f"MRR@5 on dev set: {results_dev[5]}")

---- Re-Ranking Finetune: ColBERT (SciBERT) ----
MRR@5 on dev set: 0.6532380952380952


## Results documentation

### 1) SciBERT in ColBERT architecture
Re-Ranking of top 50 BM25 results for each query:
- SciBERT (ColBERT)
    - used SciBERT model out of the box
- SciBERT (ColBERT) @ 2EP
    - fine tuned SciBERT in 2 epochs on train data
- SciBERT (ColBERT) @ 4EP
    - based on previous fine tune (SciBERT (ColBERT) @ 2EP), fine tuned SciBERT in additional 2 epochs, resulting in 4 epochs fine tune compared to out of the box SciBERT
- SciBERT (ColBERT) @ 8EP
    - based on previous fine tune (SciBERT (ColBERT) @ 4EP), fine tuned SciBERT in additional 4 epochs, resulting in 8 epochs fine tune compared to out of the box SciBERT

Hyperparameters:
- BATCH_SIZE = 8
- LR = 2e-5
- MARGIN = 0.2

Loss Function: <br>
loss = F.relu(MARGIN + score_neg_batch - score_pos_batch).mean()
<br>
<br>

|Model                        | MRR@5 (dev)     |
|-----------------------------|-----------------|
|BM25 (baseline)              |55.20%           |
|SciBERT (ColBERT)            |56.94%           | 
|SciBERT (ColBERT) @ 2EP      |63.51%           |
|SciBERT (ColBERT) @ 4EP      |64.02%           |
|SciBERT (ColBERT) @ 8EP      |62.26%           |

#### 1.1) Grid search for finding best margin and number of negative samples
| Margin | Negatives | MRR@5     | Last Epoch Loss |
|--------|-----------|-----------|--------------|
| 0.3    | 1         | 0.6537    | 8.0212       |
| 0.3    | 2         | 0.6477    | 16.7373      |
| 0.3    | 4         | 0.6118    | 29.3307      |
| 0.4    | 1         | 0.6574    | 12.9181      |
| 0.4    | 2         | 0.6503    | 21.4273      |
| 0.4    | 4         | 0.6360    | 27.8490      |
| **0.5**    | **1**         | **0.6610**    | **9.6190**       |
| 0.5    | 2         | 0.6393    | 18.1380      |
| 0.5    | 4         | 0.6259    | 31.5070      |

For margin 0.6 MRR@5 goes down again: 0.6532

# 5) Exporting results to prepare the submission on Codalab

In [21]:
model_name = "bm25"

df_query_dev['preds'] = df_query_dev[f'{model_name}_topk'].apply(lambda x: x[:5])

In [22]:
df_query_dev[['post_id', 'preds']].to_csv('predictions.tsv', index=None, sep='\t')

## _) Archive

### Grid search for best parameters

Trying out different combinations of margin and number of negative document samples for training. <br>
Result: **margin of 0.5** seems to work best.

In [11]:
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance

In [69]:
from itertools import product
import random, torch, numpy as np

param_grid = {
    "margin":        [0.3, 0.4, 0.5],
    "num_negatives": [1, 2, 4]
}
keys, values = zip(*param_grid.items())

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

for i, (margin, num_neg) in enumerate(product(*values), 1):
    set_seed()
    model_name = f"colB_sciB_m{str(margin)[-1]}_neg{num_neg}"
    print(f"\n🔍 Combination {i}/9 → {model_name}")

    scibert_finetune(
        save_name      = model_name,
        MARGIN         = margin,
        BATCH_SIZE     = 8,
        EPOCHS         = 6,
        LR             = 2e-5,
        num_negatives  = num_neg
    )

    # ---------- Evaluation ----------
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model     = AutoModel.from_pretrained(model_name)
    metadata  = pre_compute_embeddings(model_name)

    df_query_dev = rerank(df_query_dev, metadata, tokenizer, model, model_name)
    mrr_scores   = get_performance_mrr(df_query_dev, "cord_uid", f"{model_name}_topk")
    print("MRR@5:", mrr_scores[5])



🔍 Combination 1/9 → colB_sciB_m3_neg1


Epoch 1:   0%|          | 0/1607 [00:00<?, ?it/s]

Epoch 1 Loss: 121.9535


Epoch 2:   0%|          | 0/1607 [00:00<?, ?it/s]

Epoch 2 Loss: 40.3961


...

Epoch 6:   0%|          | 0/6427 [00:00<?, ?it/s]

Epoch 6 Loss: 31.5070


  0%|          | 0/7718 [00:00<?, ?it/s]

  0%|          | 0/1400 [00:00<?, ?it/s]

MRR@5: 0.6259285714285714


### 3.1) Baseline: ColBERT architecture with SciBERT
Use a pretrained SciBERT model to:
- embed each query-token
- embed each doc-token (can be pre-computed)

For each query-doc pair:
- calculate match-matrix: each query-token – doc-token pair gets cosine similarity value
- aggregate the score: 
    - for each query-token take max cosine similarity value with corresponding doc-tokens
    - sum over all of the max elements

In [14]:
def get_token_embeddings(text, tokenizer, model, device='cpu'):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model(**inputs)
    token_embeddings = outputs.last_hidden_state.squeeze(0)
    attention_mask = inputs['attention_mask'].squeeze(0).bool()
    token_embeddings = token_embeddings[attention_mask] 
    return token_embeddings

def build_and_save_doc_embeddings(
    docs_df,
    model_name,
    save_dir,
    max_len=512,
    device="cuda"
):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name).to(device)
    model.eval()

    save_path = Path("doc_embeddings_" + save_dir)
    save_path.mkdir(parents=True, exist_ok=True)

    metadata_path = save_path / "metadata.json"
    if metadata_path.exists():
        with open(metadata_path, "r") as f:
            metadata = json.load(f)
    else:
        metadata = {}

    for i, row in tqdm(docs_df.iterrows(), total=len(docs_df)):
        doc_id = row.get("cord_uid", f"doc_{i}")
        file_path = save_path / f"{doc_id}.pt"

        if file_path.exists() and doc_id in metadata:
            continue

        text = str(row.get('title', '')) + " " + str(row.get('abstract', '')) + " Authors: " + str(row.get('authors', ''))

        inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=max_len)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        output = model(**inputs)
        token_embeddings = output.last_hidden_state.squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0).bool()
        token_embeddings = token_embeddings[attention_mask]

        n_tokens = token_embeddings.size(0)
        pad_len = max_len - n_tokens

        if pad_len > 0:
            padding = torch.zeros(pad_len, token_embeddings.size(1), device=device)
            token_embeddings = torch.cat([token_embeddings, padding], dim=0)
        else:
            token_embeddings = token_embeddings[:max_len]

        try:
            torch.save(token_embeddings.cuda(), file_path)
        except Exception as e:
            print(f"Fehler beim Speichern von {doc_id}: {e}")
            continue

        metadata[doc_id] = {
            "title": row.get("title", ""),
            "abstract": row.get("abstract", ""),
            "authors": row.get("authors", ""),
            "length": min(n_tokens, max_len),
            "path": str(file_path)
        }

    with open(metadata_path, "w") as f:
        json.dump(metadata, f)

    return metadata


In [15]:
def pre_compute_embeddings(save_name):
    if not os.path.exists("doc_embeddings_" + save_name + "/metadata.json"):
        metadata = build_and_save_doc_embeddings(df_collection, model_name=model_name, save_dir=save_name, device="cuda")
    else:
        with open("doc_embeddings_" + save_name + "/metadata.json", "r") as f:
            metadata = json.load(f)
    return metadata

In [16]:
def rerank(df, metadata, tokenizer, model, save_name):
    device = next(model.parameters()).device
    df[save_name + '_scores'] = [[] for _ in range(len(df))]

    doc_embeddings = {}
    for doc_id, data in metadata.items():
        emb = torch.load(data["path"], map_location="cpu")
        doc_embeddings[doc_id] = emb

    with torch.no_grad():
        for idx, row in tqdm(df.iterrows(), total=len(df)):
            tweet_text = row['tweet_text']
            pre_ranked_docs = row['bm25_topk']

            q_emb = get_token_embeddings(tweet_text, tokenizer, model).to(device)
            q_norm = q_emb / q_emb.norm(dim=1, keepdim=True)

            scores = []
            for doc in pre_ranked_docs:
                emb = doc_embeddings[doc].to(device)
                length = metadata[doc]["length"]
                d_emb = emb[:length]
                d_norm = d_emb / d_emb.norm(dim=1, keepdim=True)

                sim_matrix = torch.matmul(q_norm, d_norm.T)
                max_sim_per_q = sim_matrix.max(dim=1).values
                score = max_sim_per_q.sum().item()
                scores.append(score)

            df.at[idx, save_name + '_scores'] = scores

    def sort_docs_by_score(row):
        doc_ids = row['bm25_topk']
        scores = row[save_name + '_scores']
        sorted_docs = [doc for doc, _ in sorted(zip(doc_ids, scores), key=lambda x: x[1], reverse=True)]
        return sorted_docs

    df[save_name + '_topk'] = df.apply(sort_docs_by_score, axis=1)
    return df

In [17]:
# settings for model run:
model_name = "allenai/scibert_scivocab_uncased"
save_name = "scibert_baseline"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/opt/conda/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [18]:
# pre-compute embeddings
metadata = pre_compute_embeddings(save_name)

In [ ]:
# re-rank BM25 list for dev data
df_query_dev = rerank(df_query_dev, metadata, tokenizer, model, save_name)

In [ ]:
# re-rank BM25 list for train data
df_query_train = rerank(df_query_train, metadata, tokenizer, model)

### ColBERT w/ fine-tuned SciBERT for docs and CTBERT for queries

In [23]:
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
from tqdm import tqdm
import json
import random
import torch.nn as nn

# hyperparameter
BATCH_SIZE = 8
EPOCHS = 4
LR = 2e-5
MARGIN = 0.2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# load model
query_model_name = "digitalepidemiologylab/covid-twitter-bert"
doc_model_name = "allenai/scibert_scivocab_uncased"

query_tokenizer = AutoTokenizer.from_pretrained(query_model_name)
doc_tokenizer = AutoTokenizer.from_pretrained(doc_model_name)

query_model = AutoModel.from_pretrained(query_model_name).to(DEVICE)
doc_model = AutoModel.from_pretrained(doc_model_name).to(DEVICE)

# projection to same dimensions
query_projection = nn.Linear(1024, 768).to(DEVICE)

# dataset creation
class ColBERTTripletDataset(Dataset):
    def __init__(self, df, metadata, num_negatives=1):
        self.data = []
        self.metadata = metadata
        for _, row in df.iterrows():
            query = row["tweet_text"]
            pos = row["cord_uid"]
            negatives = [doc for doc in row["bm25_topk"] if doc != pos]
            if negatives:
                for _ in range(num_negatives):
                    neg = random.choice(negatives)
                    self.data.append((query, pos, neg))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# helper functions
def get_token_embeddings(text, tokenizer, model, device='cpu'):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model(**inputs)
    token_embeddings = outputs.last_hidden_state.squeeze(0)
    attention_mask = inputs['attention_mask'].squeeze(0).bool()
    return token_embeddings[attention_mask]

def get_doc_embedding(doc_id, metadata, tokenizer, model, device):
    text = f"{metadata[doc_id]['title']} {metadata[doc_id]['abstract']} Authors: {metadata[doc_id]['authors']}"
    return get_token_embeddings(text, tokenizer, model, device)

def colbert_score_from_emb(q_emb, d_emb):
    q_norm = q_emb / q_emb.norm(dim=1, keepdim=True)
    d_norm = d_emb / d_emb.norm(dim=1, keepdim=True)
    sim_matrix = torch.matmul(q_norm, d_norm.T)
    return sim_matrix.max(dim=1).values.sum()

# training function
def train_colbert_dual_encoder(df_query_train, metadata):
    dataset = ColBERTTripletDataset(df_query_train, metadata)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

    query_model.train()
    doc_model.train()
    query_projection.train()

    optimizer = torch.optim.AdamW(
        list(query_model.parameters()) +
        list(doc_model.parameters()) +
        list(query_projection.parameters()),
        lr=LR
    )

    for epoch in range(EPOCHS):
        total_loss = 0.0
        for batch in tqdm(loader, desc=f"Epoch {epoch+1}"):
            queries, pos_ids, neg_ids = batch

            inputs = query_tokenizer(list(queries), return_tensors='pt', padding=True, truncation=True, max_length=512)
            inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
            outputs = query_model(**inputs)
            q_emb_batch = outputs.last_hidden_state
            attention_mask = inputs["attention_mask"].bool()
            q_embs = [query_projection(emb[mask]) for emb, mask in zip(q_emb_batch, attention_mask)]

            score_pos_list = []
            score_neg_list = []

            for i in range(len(queries)):
                d_pos_emb = get_doc_embedding(pos_ids[i], metadata, doc_tokenizer, doc_model, DEVICE)
                d_neg_emb = get_doc_embedding(neg_ids[i], metadata, doc_tokenizer, doc_model, DEVICE)
                q_emb = q_embs[i]

                score_pos = colbert_score_from_emb(q_emb, d_pos_emb)
                score_neg = colbert_score_from_emb(q_emb, d_neg_emb)

                score_pos_list.append(score_pos)
                score_neg_list.append(score_neg)

            score_pos_batch = torch.stack(score_pos_list)
            score_neg_batch = torch.stack(score_neg_list)

            loss = F.relu(MARGIN + score_neg_batch - score_pos_batch).mean()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_loss += loss.item()

        print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

    query_model.save_pretrained("finetuned_ctbert")
    query_tokenizer.save_pretrained("finetuned_ctbert")
    doc_model.save_pretrained("finetuned_scibert")
    doc_tokenizer.save_pretrained("finetuned_scibert")
    torch.save(query_projection.state_dict(), "query_projection.pt")


In [ ]:
train_colbert_dual_encoder(df_query_train, metadata)

In [32]:
def rerank(df, metadata, tokenizer, q_model, d_model, save_name):
    device_q = next(q_model.parameters()).device
    device_d = next(d_model.parameters()).device
    df[save_name + '_scores'] = [[] for _ in range(len(df))]

    query_projection = nn.Linear(1024, 768)
    query_projection.load_state_dict(torch.load("query_projection.pt"))
    query_projection.to(device_q)
    query_projection.eval()

    doc_embeddings = {}
    for doc_id, data in metadata.items():
        emb = torch.load(data["path"], map_location="cpu")
        doc_embeddings[doc_id] = emb

    with torch.no_grad():
        for idx, row in tqdm(df.iterrows(), total=len(df)):
            tweet_text = row['tweet_text']
            pre_ranked_docs = row['bm25_topk']

            q_emb = get_token_embeddings(tweet_text, tokenizer, q_model, device=device_q)
            q_emb = query_projection(q_emb)
            q_norm = q_emb / q_emb.norm(dim=1, keepdim=True)

            scores = []
            for doc in pre_ranked_docs:
                emb = doc_embeddings[doc].to(device_d)
                length = metadata[doc]["length"]
                d_emb = emb[:length]
                d_norm = d_emb / d_emb.norm(dim=1, keepdim=True)

                sim_matrix = torch.matmul(q_norm, d_norm.T)
                max_sim_per_q = sim_matrix.max(dim=1).values
                score = max_sim_per_q.sum().item()
                scores.append(score)

            df.at[idx, save_name + '_scores'] = scores

    def sort_docs_by_score(row):
        doc_ids = row['bm25_topk']
        scores = row[save_name + '_scores']
        sorted_docs = [doc for doc, _ in sorted(zip(doc_ids, scores), key=lambda x: x[1], reverse=True)]
        return sorted_docs

    df[save_name + '_topk'] = df.apply(sort_docs_by_score, axis=1)
    return df

In [25]:
# pre-compute embeddings
metadata = pre_compute_embeddings("finetuned_scibert")

100%|██████████| 7718/7718 [03:40<00:00, 34.94it/s]


In [33]:
# re-rank BM25 list for dev data
df_query_dev = rerank(df_query_dev, metadata, tokenizer, query_model, doc_model, "sciCtBERT-1")

100%|██████████| 1400/1400 [02:21<00:00,  9.89it/s]
